# MSA

The MSA module of MIToS has utilities for working with Multiple Sequence Alignments of protein Sequences (MSA).

## Features

- [Read and write MSAs](#Reading-and-writing-MSA-files.) in `Stockholm`, `FASTA` or `Raw` format
- Handle MSA annotations
- Edit the MSA, e.g. delete columns or sequences, change sequence order, shuffling...
- Keep track of positions and annotations after modifications on the MSA
- Describe a MSA, e.g. mean percent identity, sequence coverage, gap percentage...

In [5]:
using MIToS.MSA

## Reading and writing MSA files.

The main function for reading files in MIToS is `read` and it is defined in the `Utils` module. This function takes a filename/path and lot of arguments, opens the file and uses the arguments to call the `parse` function. `read` decides how to open the file, using the prefixes and suffixes of the file name, while `parse` does the actual parsing of the file. You can `read` **gzipped files** if they have the `.gz` extension and also **files of the web**.  
The second argument of `read` and `parse` is the file `Format`. The supported MSA formats at the moment are `Stockholm`, `FASTA` and `Raw`.  
For example, reading in Julia the full Stockholm MSA of the family PF07388 using the Pfam RESTful interface will be:

In [10]:
msa = read("http://pfam.xfam.org/family/PF07388/alignment/full", Stockholm)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4168  100  4168    0     0   7883      0 --:--:-- --:--:-- --:--:--  7893


4x458 MIToS.MSA.AnnotatedMultipleSequenceAlignment:
 -  -  -  -  -  -  -  -  -  -  -  -  -  …  -  -  -  -  -  -  -  -  -  -  -  -
 M  L  K  K  I  K  K  A  L  F  Q  P  K     -  -  -  -  -  -  -  -  -  -  -  -
 -  -  K  K  L  S  G  L  M  Q  D  I  K     D  F  Q  K  Y  R  I  K  Y  L  Q  L
 -  -  -  -  -  -  -  -  -  -  -  -  -     -  -  -  -  -  -  -  -  -  -  -  -

The third (and optional) argument of `read` and `parse` is the output MSA type:  
  
<p>
    <dl>
    
        <dt><code>Matrix{Residue}</code></dt>
        <dd>It is the default output format for a <code>Raw</code> file.</dd>
        
        <dt><code>MultipleSequenceAlignment</code></dt>
        <dd>It contains the sequence names/identifiers.</dd>
        
        <dt><code>AnnotatedMultipleSequenceAlignment</code></dt>
        <dd>The richest MSA format of MIToS and the default for <code>FASTA</code> and <code>Stockholm</code> files. It includes sequences names and MSA annotations.</dd>
       
    </dl>
</p>


In [18]:
msa = read("http://pfam.xfam.org/family/PF07388/alignment/full", Stockholm, Matrix{Residue})

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4168  100  4168    0     0   7389      0 --:--:-- --:--:-- --:--:--  7390


4x458 Array{MIToS.MSA.Residue,2}:
 -  -  -  -  -  -  -  -  -  -  -  -  -  …  -  -  -  -  -  -  -  -  -  -  -  -
 M  L  K  K  I  K  K  A  L  F  Q  P  K     -  -  -  -  -  -  -  -  -  -  -  -
 -  -  K  K  L  S  G  L  M  Q  D  I  K     D  F  Q  K  Y  R  I  K  Y  L  Q  L
 -  -  -  -  -  -  -  -  -  -  -  -  -     -  -  -  -  -  -  -  -  -  -  -  -

Given that `read` call `parse`, you should look into the documentation of the last one to know the available keyword arguments. The optional keyword arguments using in MSA IO are:

<p>
<dl>

<dt><code>generatemapping</code></dt>
<dd>
If <code>checkalphabet</code> is <code>true</code> (default to <code>false</code>), sequence and columns mapping are generated and saved in the MSA annotations. <span class="text-warning">The default is <code>false</code> to not overwrite mappings by mistake when you read an annotated MSA file saved with MIToS.</span>
</dd>

<dt><code>useidcoordinates</code></dt>
<dd>
If <code>useidcoordinates</code> is <code>true</code> (default to <code>false</code>), MIToS uses the coordinates in the sequence names of the form <i>seqname/start-end</i> to generate sequence mappings. This is safe and useful with fresh downloaded Pfam MSAs. <span class="text-warning">Please be careful if you are reading a MSA saved with MIToS. MIToS deletes unaligned insert columns, therefore the sequences would be disrupted if there were insert columns.</span>
</dd>

<dt><code>checkalphabet</code></dt>
<dd>
The <code>parse</code> function converts each character in the sequence strings to a MIToS <code>Residue</code>. Lowercase characters and degenerated or non standard residues are converted to gaps. If <code>checkalphabet</code> is <code>true</code> (<code>false</code> by default), <code>read</code> deletes all the sequences with non-standard residues. The 20 natural residues are A, R, N, D, C, Q, E, G, H, I, L, K, M, F, P, S, T, W, Y and V.
</dd>

<dt><code>deletefullgaps</code></dt>
<dd>
Given that lowercase characters and dots are converted to gaps, unaligned insert columns in a MSA derived from a HMM profile are converted into full gap columns. <code>deletefullgaps</code> is <code>true</code> by default, deleting full gaps columns and therefore insert columns.
</dd>

</dl>
</p>

<div class="panel panel-warning">
<div class="panel-heading">
		<strong>If you are deriving scores from gaps...</strong>
	</div>
	<div class="panel-body">
		If you are using MIToS to derive information scores from gaps, you will want to set <code>checkalphabet</code> to <code>true</code>. This prevents counting non standard residues as gaps.
	</div>
</div>

In [4]:
?MIToS.MSA.@res_str

#### res"..."

The MIToS macro `@res_str` takes a string and returns a `Vector` of `Residues` (sequence).

```julia

julia> res"MIToS"
5-element Array{MIToS.MSA.Residue,1}:
 M
 I
 T
 -
 S

```


<div class="panel panel-info">
    <div class="panel-heading">
        <strong>Julia help mode</strong>
    </div>
    <div class="panel-body">
        <p>If you type <code>?</code> at the beginning of the Julia REPL line, you will enter in the Julia help mode. In this mode, Julia prints the help or <strong>documentation</strong> of the entered element. This is a nice way of getting information about MIToS functions, types, etc. from Julia.</p>
    </div>
</div>

# Multiple Sequence Alignments

In [5]:
msa_file = MIToS.Pfam.downloadpfam("PF09645")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0

"PF09645.stockholm.gz"

100   647  100   647    0     0    280      0  0:00:02  0:00:02 --:--:--   280


In [6]:
?MIToS.MSA.AbstractMultipleSequenceAlignment

### AbstractMultipleSequenceAlignment

The most basic implementation of a MIToS MSA is a `Matrix` of `Residue`s.


In [7]:
msa = read(msa_file, Stockholm, Matrix{Residue})

2x110 Array{MIToS.MSA.Residue,2}:
 -  -  -  -  -  -  -  V  A  Q  Q  L  F  …  -  -  -  -  -  -  -  -  -  -  -  -
 Q  T  L  N  S  Y  K  M  A  E  I  M  Y     E  Q  T  D  Q  G  F  I  K  A  K  Q

In [8]:
?MIToS.MSA.MultipleSequenceAlignment

### MultipleSequenceAlignment

This MSA type include the `Matrix` of `Residue`s and the sequence names. To allow fast indexing of MSAs using **sequence identifiers**, they are saved as an `IndexedArray`.


In [ ]:
msa = read(msa_file, Stockholm, MultipleSequenceAlignment)

In [ ]:
msa.id

In [ ]:
msa["F112_SSV1/3-112"]

Similar to this, MIToS defines an `AnnotatedMultipleSequenceAlignment` that also includes annotations.

In [ ]:
fieldnames(AnnotatedMultipleSequenceAlignment)

In [ ]:
msa = read(msa_file, Stockholm, AnnotatedMultipleSequenceAlignment, generatemapping=true, useidcoordinates=true)

In [ ]:
msa.annotations

## MSA annotations

In [ ]:
?MIToS.MSA.Annotations

In [ ]:
fieldnames(Annotations)

MIToS uses MSA annotations to keep track of:  
- **Modifications** of the MSA (`MIToS_...`) as deletion of sequences or columns.  
- Positions numbers in the original MSA file (**column mapping:** `ColMap`)  
- Position of the residues in the sequence (**sequence mapping:** `SeqMap`)  

In [ ]:
printmodifications(msa)

In [ ]:
getcolumnmapping(msa)

In [ ]:
getsequencemapping(msa,"F112_SSV1/3-112")